# Working with the NPU driver

![](images/xdna_arch.png)

A Neural Processing Unit (NPU) is a specialized processing unit designed to excel in Machine Learning inference tasks. An NPU boosts the speed and efficiency of machine learning tasks on your computer while consuming significantly less power than traditional CPU or GPU approaches. 

The NPU is another accelerator with its own set of tools and utilities, which are distinct but similar to existing utilities you may be familiar with. In this notebook we'll walk through how to use the utilities that get shipped with the XDNA driver, which help us inspect the status of the device.

## Goals

* Get to know the XDNA driver
* Learn to use `xrt-smi` to check status
* Validate your device before moving on

## References

* [XDNA Driver Repo](https://github.com/amd/xdna-driver)
* [NPU Management Docs](https://ryzenai.docs.amd.com/en/latest/xrt_smi.html)
* [AMD_XDNA](https://en.wikipedia.org/wiki/AMD_XDNA)

## The XDNA Driver

The NPU is based on the XDNA microarchitecture. This architecture contains an AI Engine (AIE) array composed of dense compute cores tuned for Machine Learning (ML) workload operations like convolutions and matrix multiplications. In order to program this device we need to utilize a kernel-level driver `amdxdna.ko`.

We can actually check if the driver is loaded using `lsmod`:

In [1]:
!sudo lsmod | grep amdxdna

amdxdna               335872  0
gpu_sched              61440  2 amdxdna,amdgpu


The NPU device like many accelerators use the kernel's existing scheduling infrastructure hence it shows up under `gpu_sched`. For the purposes of running on code on this accelerator we just want to make sure that the kernel module is loaded.

## Examining the Device

Much like the GPU where you have utilities like `rocm-smi` to view the status and resource usage, when we work with the NPU we use `xrt-smi`. Here's a quick rundown of the options for this utility:

In [2]:
!xrt-smi --help


DESCRIPTION: The XRT - System Management Interface (xrt-smi) is a standalone command-line utility
             that is included with the XRT runtime package. It includes multiple commands to
             configure, examine, and validate supported device(s).
             
             The reports produced by xrt-smi may be used for device administration, monitoring, and
             troubleshooting application behavior.

USAGE: xrt-smi [--help] [--version] [--verbose] [--batch] [--force] [command [commandArgs]]

AVAILABLE COMMANDS:
  configure  - Device and host configuration
  examine    - Status of the system and device
  validate   - Validates the basic device acceleration functionality

OPTIONS:
  --help             - Help to use this application
  --version          - Report the version of XRT and its drivers
  --verbose          - Turn on verbosity
  --batch            - Enable batch mode (disables escape characters)
  --force            - When possible, force an operation


Most commonly you will want to use the `examine` command. This will give you a system overview as well as driver and firmware info -- crucial when submitting bug reports and debugging!

In [3]:
!xrt-smi examine

System Configuration
  OS Name              : Linux
  Release              : 6.14.0-33-generic
  Machine              : x86_64
  CPU Cores            : 32
  Memory               : 31725 MB
  Distribution         : Ubuntu 24.04.3 LTS
  GLIBC                : 2.39
  Model                : NucBox_EVO-X2
  BIOS Vendor          : American Megatrends International, LLC.
  BIOS Version         : EVO-X2 1.05

XRT
  Version              : 2.20.0
  Branch               : HEAD
  Hash                 : 021204355eeaa034ff69aae407ace2265adf047a
  Hash Date            : 2025-10-07 17:47:22
  amdxdna              : 2.20.0_20251007, 0ad5aa3765bb969135df1ae8ee26dae0bf65559a
  virtio-pci           : unknown, unknown
  NPU Firmware Version : 255.0.4.33

Device(s) Present
|BDF             |Name            |
|----------------|----------------|
|[0000:c6:00.1]  |NPU Strix Halo  |




The hashes correspond to commits the driver was built with - this is all open source and you can always update your driver to the bleeding edge by building from https://github.com/amd/xdna-driver.

We can scope down and report only the platform information about the NPU device.

In [4]:
!xrt-smi examine --report platform


--------------------------------
[0000:c6:00.1] : NPU Strix Halo
--------------------------------
Platform
  Name                   : NPU Strix Halo 
  Power Mode             : Default 
  Total Columns          : 8 

Estimated Power          : N/A



Note that some features like power estimation may not be available on all platforms, but this is a useful probe to check the device class, how big it is and whether the power mode has been changed.

## Reporting Hardware Contexts

This is a super important command because it will give you a populated list of running NPU applications. Sometimes when developing on the NPU or if you download an application that is meant to run on this platform you will want to know if it is **actually** running on the NPU. How can you tell?

**Hardware contexts** are spatial allocations on the NPU array for a given application. The XDNA driver and NPU firmware work together to provision a spatial partition on the NPU's array, dedicating a set of columns for the workload.

To examine these allocations we use the `--report aie-partitions` suboption.

In [5]:
!xrt-smi examine --report aie-partitions


--------------------------------
[0000:c6:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  No hardware contexts running on device



If nothing is running on the NPU this is the expected output. Additionally you can also use a handy all-in-one `--report all` suboption, to report everything! Again, knowing the system state is very useful for debugging.

In [6]:
!xrt-smi examine --report all

System Configuration
  OS Name              : Linux
  Release              : 6.14.0-33-generic
  Machine              : x86_64
  CPU Cores            : 32
  Memory               : 31725 MB
  Distribution         : Ubuntu 24.04.3 LTS
  GLIBC                : 2.39
  Model                : NucBox_EVO-X2
  BIOS Vendor          : American Megatrends International, LLC.
  BIOS Version         : EVO-X2 1.05

XRT
  Version              : 2.20.0
  Branch               : HEAD
  Hash                 : 021204355eeaa034ff69aae407ace2265adf047a
  Hash Date            : 2025-10-07 17:47:22
  amdxdna              : 2.20.0_20251007, 0ad5aa3765bb969135df1ae8ee26dae0bf65559a
  virtio-pci           : unknown, unknown
  NPU Firmware Version : 255.0.4.33

Device(s) Present
|BDF             |Name            |
|----------------|----------------|
|[0000:c6:00.1]  |NPU Strix Halo  |



--------------------------------
[0000:c6:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  No hardware 

## Validating device

OK now that we know that the NPU device exists and we can probe it for more info let's validate that the driver works as intended in our environment. We can use the `validate` subcommand for this. It will run a series of standard tests, and if everything passes we can be fairly confident that our device is functioning as intended.

In [7]:
!xrt-smi validate --batch

Validate Device           : [0000:c6:00.1]
    Platform              : NPU Strix Halo
    Power Mode            : Default
-------------------------------------------------------------------------------
Running Test: ..
Test 1 [0000:c6:00.1]     : gemm 
    Details               : TOPS: 51.0
    Test Status           : [PASSED]
-------------------------------------------------------------------------------
Running Test: .
Test 2 [0000:c6:00.1]     : latency 
    Details               : Average latency: 48.0 us
    Test Status           : [PASSED]
-------------------------------------------------------------------------------
Running Test: ..
Test 3 [0000:c6:00.1]     : throughput 
    Details               : Average throughput: 80247.0 op/s
    Test Status           : [PASSED]
-------------------------------------------------------------------------------
Validation completed. Please run the command '--verbose' option for more details


Excellent, your NPU device is operational and the driver is set up accordingly!

## Examine NPU Usage

Now that we have something that runs on the NPU - let's use what we learned to examine what a running application hardware context looks like.

We'll create a subprocess to run our validation tests. And while that is running let's examine the aie-partitions report.

In [9]:
import subprocess
import time

# Run the command in the background
process = subprocess.Popen(
    ['xrt-smi', 'validate'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print(f"Process started with PID: {process.pid}")

time.sleep(1) # give it a second for contexts to get populated
!xrt-smi examine --report aie-partitions

Process started with PID: 288

--------------------------------
[0000:c6:00.1] : NPU Strix Halo
--------------------------------
AIE Partitions
  Total Memory Usage: N/A
  Partition Index   : 0
    Columns: [0, 1, 2, 3, 4, 5, 6, 7]
    HW Contexts:
      |PID                 |Ctx ID     |Submissions |Migrations  |Err  |Priority |
      |Process Name        |Status     |Completions |Suspensions |     |GOPS     |
      |Memory Usage        |Instr BO   |            |            |     |FPS      |
      |                    |           |            |            |     |Latency  |
      |====================|===========|============|============|=====|=========|
      |145953              |1          |149         |0           |0    |Normal   |
      |N/A                 |Active     |148         |0           |     |N/A      |
      |N/A                 |2476 KB    |            |            |     |N/A      |
      |                    |           |            |            |     |N/A      |
    

Since the tests are utilizing the full array we see all 8 columns being used. Once we run our own applications, if we see this hw context table being populated we can be confident that the NPU is being used.

## Next Steps

Now that we know how utilities like `xrt-smi` work and how to use them to check NPU status and debug, let's move onto more practical programming exercises.

**Continue to**: [2_ryzenai_cvml_library.ipynb](2_ryzenai_cvml_library.ipynb)

---
Copyright© 2025 AMD, Inc SPDX-License-Identifier: MIT